# Dataset factory

Objetivo: crear un dataset "rellenado" de estados de los cajeros encodeado por estatus (verde = disponible, rojo = no disponible).

algo así:


| ATM_ID            | fecha (dia)   | status | nombre_indisponibilidad | status_encodeado
|-------------------|-------------|---------------|---------------|---------------|
| 1   | 22-10-22    | rojo           | RA1 Power Failure | -1 |
| 1       | 24-10-22       | rojo             | RA1 Power Failure | -1 | 
| 1         | 25-10-22  | verde            | None | 1 |
| 1         | 27-10-22  | verde            | None | 1 |
| 1         | 28-10-22  | naranja           | Advertencia | 0 |  
| 1         | 30-10-22  | rojo            | RA1 Power Failure | -1 |

Otras columnas que vale la pena conservar/mapear para poder usarlo en el dashboard:

- tipo de dispositivo (atms_raw) [x]
- tipo_sitio "SUC" o "REM" (atms_raw o issues_raw) [x] 
- latitud (atms_raw) [x]
- longitud (atms_raw) [x]

Incluir solo fallas que hayan durado más de un día y menos de 1 mes (Antes de limpiar, graficar por año para tener idea de los outliers que hay)

In [2]:
import pandas as pd

In [3]:
issues_raw = pd.read_csv("data_hackaton/issues-raw.csv", sep=";", decimal=",")

Escoger de las columnas disponibles, solo las que nos interesan, enfocarnos primero en el dataset de issues

In [4]:
print(issues_raw.columns)

Index(['ATM_ID', 'SITIO', 'division', 'TIPO_SITIO', 'MARCA_MODELO', 'FALLA',
       'FECHA_INICIO', 'FECHA_FIN', 'DURACION', 'IMPACTO', 'TICKET_KEY',
       'ESTADO', 'CR', 'AUTOSERVICIO', 'NAME', 'faultid', 'generacion'],
      dtype='object')


In [5]:
issues_cooked = issues_raw[['ATM_ID', 'FECHA_INICIO', 'FECHA_FIN', 'FALLA', 'TIPO_SITIO']]
issues_cooked.head(2)

,ATM_ID,FECHA_INICIO,FECHA_FIN,FALLA,TIPO_SITIO
0,1131,23/8/22 12:17,23/8/22 12:46,P2F C Reset por Telecontrol,REM
1,2464,23/8/22 12:20,23/8/22 12:49,RA1 Power Failure,REM


Obtener una muestra de una serie para un solo cajero, como prueba del algoritmo

In [6]:
issues_cooked['ATM_ID'].value_counts()

448     376
E901    250
5328    241
8431    211
5276    195
       ... 
A516      1
8698      1
9582      1
C263      1
5037      1
Name: ATM_ID, Length: 12533, dtype: int64

El cajero 448 es el que tiene más ocurrencias, hay algunos en el set que solo tienen un registro de falla, **probablemente sea inteligente filtrar los cajeros con bajas ocurrencias de fallas**. Por ahora escogeré el cajero 448 como sujeto de prueba

In [7]:
sample_issues = issues_cooked[issues_cooked['ATM_ID'] == "448"].copy()
sample_issues.head(2)

,ATM_ID,FECHA_INICIO,FECHA_FIN,FALLA,TIPO_SITIO
30972,448,20/8/22 01:19,20/8/22 01:41,Fuera de Sesion RA1 ...,REM
34207,448,12/8/22 08:00,12/8/22 08:03,Fuera de sesion (atn mto tec),REM


Listo, con el cajero aislado ahora puedo hacer un ordenamiento por fecha de inicio, sería imposible que un cajero reportara una falla antes de que se haya resuelto la anterior

In [13]:
sample_issues.sort_values(by="FECHA_INICIO", ascending=False)

,ATM_ID,FECHA_INICIO,FECHA_FIN,FALLA,TIPO_SITIO
45995,448,9/8/22 23:55,9/8/22 23:58,P2F 7 Falla en Comunicaciones,REM
45994,448,9/8/22 23:45,9/8/22 23:48,P2F 7 Falla en Comunicaciones,REM
45993,448,9/8/22 23:35,9/8/22 23:37,P2F 7 Falla en Comunicaciones,REM
45956,448,9/8/22 23:23,9/8/22 23:27,P2F 7 Falla en Comunicaciones,REM
45954,448,9/8/22 23:13,9/8/22 23:16,P2F 7 Falla en Comunicaciones,REM
...,...,...,...,...,...
46004,448,10/8/22 00:59,10/8/22 01:01,P2F 7 Falla en Comunicaciones,REM
46003,448,10/8/22 00:43,10/8/22 00:50,P2F 7 Falla en Comunicaciones,REM
46000,448,10/8/22 00:33,10/8/22 00:35,P2F 7 Falla en Comunicaciones,REM
45998,448,10/8/22 00:17,10/8/22 00:25,P2F 7 Falla en Comunicaciones,REM


In [14]:
sample_issues.columns

Index(['ATM_ID', 'FECHA_INICIO', 'FECHA_FIN', 'FALLA', 'TIPO_SITIO'], dtype='object')

In [16]:
sample_issues.melt(id_vars=['ATM_ID', 'FALLA', 'TIPO_SITIO'],
                   value_vars=['FECHA_INICIO', 'FECHA_FIN'],
                   var_name="TIPO_FECHA",
                   value_name="TIMESTAMP").sort_values(by="TIMESTAMP", ascending=False)

,ATM_ID,FALLA,TIPO_SITIO,TIPO_FECHA,TIMESTAMP
522,448,P2F 7 Falla en Comunicaciones,REM,FECHA_FIN,9/8/22 23:58
146,448,P2F 7 Falla en Comunicaciones,REM,FECHA_INICIO,9/8/22 23:55
521,448,P2F 7 Falla en Comunicaciones,REM,FECHA_FIN,9/8/22 23:48
145,448,P2F 7 Falla en Comunicaciones,REM,FECHA_INICIO,9/8/22 23:45
520,448,P2F 7 Falla en Comunicaciones,REM,FECHA_FIN,9/8/22 23:37
...,...,...,...,...,...
149,448,P2F 7 Falla en Comunicaciones,REM,FECHA_INICIO,10/8/22 00:33
524,448,P2F 7 Falla en Comunicaciones,REM,FECHA_FIN,10/8/22 00:25
148,448,P2F 7 Falla en Comunicaciones,REM,FECHA_INICIO,10/8/22 00:17
523,448,P2F 7 Falla en Comunicaciones,REM,FECHA_FIN,10/8/22 00:09


In [102]:
def df_transformer(df):
    # Melt the dates, sort them from earliest to latest by atm. Remove duplicated events for the same ATM
    df = df.melt(id_vars=['ATM_ID', 'FALLA', 'TIPO_SITIO'],
                    value_vars=['FECHA_INICIO', 'FECHA_FIN'],
                    var_name="TIPO_FECHA",
                    value_name="TIMESTAMP")
    df = df.sort_values(by=["ATM_ID", "TIMESTAMP"], ascending=False)
    df = df.drop_duplicates(subset=['ATM_ID', 'TIMESTAMP'])
    # Fill forward between "FECHA_FIN" and "FECHA_INICIO" values
    
    return df

In [103]:
sample_melted = df_transformer(sample_issues)

In [101]:
pd.to_datetime(sample_melted.iloc[1]['TIMESTAMP']) - pd.to_datetime(sample_melted.iloc[0]['TIMESTAMP'])

Timedelta('-1 days +23:57:00')

In [86]:
sample_melted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 147 to 522
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ATM_ID      752 non-null    object
 1   FALLA       752 non-null    object
 2   TIPO_SITIO  752 non-null    object
 3   TIPO_FECHA  752 non-null    object
 4   TIMESTAMP   752 non-null    object
dtypes: object(5)
memory usage: 35.2+ KB


In [87]:
sample_melted['TIMESTAMP'] = pd.to_datetime(sample_melted['TIMESTAMP'])

In [88]:
sample_melted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 147 to 522
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ATM_ID      752 non-null    object        
 1   FALLA       752 non-null    object        
 2   TIPO_SITIO  752 non-null    object        
 3   TIPO_FECHA  752 non-null    object        
 4   TIMESTAMP   752 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 35.2+ KB


In [98]:
sample_melted.iloc[600]['TIMESTAMP'] - sample_melted.iloc[0]['TIMESTAMP']

Timedelta('-30 days +07:18:00')

In [97]:
idx = pd.date_range(start=sample_melted.iloc[0]['TIMESTAMP'], end=sample_melted.iloc[600]['TIMESTAMP'])
print(idx)

DatetimeIndex([], dtype='datetime64[ns]', freq='D')


In [71]:
def date_filler(df):
    # idx = pd.date_range(pd.to_datetime(df['TIMESTAMP'].iloc[0]), pd.to_datetime(df['TIMESTAMP'].iloc[-1]))
    idx = pd.date_range('08-01-2022', '08-15-2022')
    df.index = pd.DatetimeIndex(df['TIMESTAMP'])
    df_out = df.reindex(idx, fill_value=0)

    return df_out

In [72]:
date_filler(sample_melted)

,ATM_ID,FALLA,TIPO_SITIO,TIPO_FECHA,TIMESTAMP
2022-08-01,0,0,0,0,0
2022-08-02,0,0,0,0,0
2022-08-03,0,0,0,0,0
2022-08-04,0,0,0,0,0
2022-08-05,0,0,0,0,0
2022-08-06,0,0,0,0,0
2022-08-07,0,0,0,0,0
2022-08-08,0,0,0,0,0
2022-08-09,0,0,0,0,0
2022-08-10,0,0,0,0,0


In [76]:
idx = pd.date_range('08-01-2022', '08-15-2022')
idx

DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07', '2022-08-08',
               '2022-08-09', '2022-08-10', '2022-08-11', '2022-08-12',
               '2022-08-13', '2022-08-14', '2022-08-15'],
              dtype='datetime64[ns]', freq='D')

In [73]:
idx = pd.date_range('09-01-2013', '09-30-2013')

s = pd.Series({'09-02-2013': 2,
               '09-03-2013': 10,
               '09-06-2013': 5,
               '09-07-2013': 1})
s.index = pd.DatetimeIndex(s.index)
print(s)


2013-09-02     2
2013-09-03    10
2013-09-06     5
2013-09-07     1
dtype: int64


In [74]:
idx

DatetimeIndex(['2013-09-01', '2013-09-02', '2013-09-03', '2013-09-04',
               '2013-09-05', '2013-09-06', '2013-09-07', '2013-09-08',
               '2013-09-09', '2013-09-10', '2013-09-11', '2013-09-12',
               '2013-09-13', '2013-09-14', '2013-09-15', '2013-09-16',
               '2013-09-17', '2013-09-18', '2013-09-19', '2013-09-20',
               '2013-09-21', '2013-09-22', '2013-09-23', '2013-09-24',
               '2013-09-25', '2013-09-26', '2013-09-27', '2013-09-28',
               '2013-09-29', '2013-09-30'],
              dtype='datetime64[ns]', freq='D')

In [75]:
s = s.reindex(idx, fill_value=0)
print(s)

2013-09-01     0
2013-09-02     2
2013-09-03    10
2013-09-04     0
2013-09-05     0
2013-09-06     5
2013-09-07     1
2013-09-08     0
2013-09-09     0
2013-09-10     0
2013-09-11     0
2013-09-12     0
2013-09-13     0
2013-09-14     0
2013-09-15     0
2013-09-16     0
2013-09-17     0
2013-09-18     0
2013-09-19     0
2013-09-20     0
2013-09-21     0
2013-09-22     0
2013-09-23     0
2013-09-24     0
2013-09-25     0
2013-09-26     0
2013-09-27     0
2013-09-28     0
2013-09-29     0
2013-09-30     0
Freq: D, dtype: int64
